In [79]:
import contextlib
from scipy.ndimage import gaussian_filter
import rasterio
import rasterio.merge
import os
from rasterio.enums import Resampling 
import geopandas

In [80]:
print(geopandas.__version__)

0.8.1


In [81]:
print(rasterio.__version__)

1.1.8


In [82]:
inpaths=['C:/Users/Rasmu/Documents/Thesis/S1_raw/VH_20200214_desc.tif']
#inpaths=['C:/Users/Rasmu/Documents/Thesis/S2_Masks/SI.tif']
outfile = 'C:/Users/Rasmu/Documents/Thesis/S1_resampled/VH_20200214_desc_S5.tif'
#outfile = 'C:/Users/Rasmu/Documents/Thesis/S2_Masks/SI_1000m.tif'

In [83]:
## RASTER MERGE ##
def merge_my_files(inpaths, outpath):
        with contextlib.ExitStack() as es:
            datasets = [es.enter_context(rasterio.open(path)) for path in inpaths]
            vrts = [es.enter_context(rasterio.vrt.WarpedVRT(ds, crs='epsg:25833')) for ds in datasets]
            profile = datasets[0].profile.copy()
            data, transform = rasterio.merge.merge(vrts, method='last', res=100,resampling = Resampling.bilinear)
            print(data.shape)            
            data = gaussian_filter(data, sigma=2)
            profile.update({
            'width': data.shape[-1],
            'height': data.shape[-2],
            'transform': transform,
            'crs': vrts[0].crs,
        })
        with rasterio.open(outpath, 'w', **profile) as dst:
            dst.write(data)

In [84]:
merge_my_files(inpaths,outfile)

(1, 1305, 1635)
